In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# загружаем и подготавляваем данные
df = pd.read_csv('/content/data.csv')

df = df.iloc[np.random.permutation(len(df))]
y = df.iloc[0:100, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[0:100, [0, 2]].values


In [5]:
inputSize = X.shape[1] # количество входных сигналов равно количеству признаков задачи
hiddenSizes = 10 # задаем число нейронов скрытого (А) слоя
outputSize = 1 if len(y.shape) else y.shape[1] # количество выходных сигналов равно количеству классов задачи

In [6]:
# создаем матрицу весов скрытого слоя
Win = np.zeros((1+inputSize,hiddenSizes))
# пороги w0 задаем случайными числами
Win[0,:] = (np.random.randint(0, 3, size = (hiddenSizes)))
# остальные веса  задаем случайно -1, 0 или 1
Win[1:,:] = (np.random.randint(-1, 2, size = (inputSize,hiddenSizes)))

In [7]:
#Wout = np.zeros((1+hiddenSizes,outputSize))

# случайно инициализируем веса выходного слоя
Wout = np.random.randint(0, 2, size = (1+hiddenSizes,outputSize)).astype(np.float64)

In [8]:
# функция прямого прохода (предсказания)
def predict(Xp):
    # выходы первого слоя = входные сигналы * веса первого слоя
    hidden_predict = np.where((np.dot(Xp, Win[1:,:]) + Win[0,:]) >= 0.0, 1, -1).astype(np.float64)
    # выходы второго слоя = выходы первого слоя * веса второго слоя
    out = np.where((np.dot(hidden_predict, Wout[1:,:]) + Wout[0,:]) >= 0.0, 1, -1).astype(np.float64)
    return out, hidden_predict

In [17]:
# обучение
# у перцептрона Розенблатта обучаются только веса выходного слоя
# как и раньше обучаем подавая по одному примеру и корректируем веса в случае ошибки
n_iter=0
eta = 0.01

# список для хранения  матрицы весов второго слоя
list_weights = [];

break_out_flag = True

#обучение
while(break_out_flag):
    print('iteration:',n_iter)
    n_iter+=1

    for x_input, expected, j in zip(X, y, range(X.shape[0])):
        W_2_out, W_1_out = predict(x_input)
        Wout[1:] += ((eta * (expected - W_2_out)) * W_1_out).reshape(-1, 1)
        Wout[0] += eta * (expected - W_2_out)

    #список матрицы весов второго слоя
    list_weights.append(Wout.tobytes())

    for item in list_weights:
        if list_weights.count(item) > 1:
            print('Повторение весов!')
            break_out_flag = False
            #break

    W_2_out, W_1_out = predict(X)
    sum_errors = sum(abs(W_2_out.reshape(-1) - y))
    print('sum_errors', sum_errors)

    if (sum_errors == 0):
        print('Все примеры обучающей выборки решены верно')
        break

for i in range(n_iter):
    for xi, target, j in zip(X, y, range(X.shape[0])):
        pr, hidden = predict(xi)
        Wout[1:] += ((eta * (target - pr)) * hidden).reshape(-1, 1)
        Wout[0] += eta * (target - pr)

iteration: 0
sum_errors 100.0
iteration: 1
Повторение весов!
Повторение весов!
sum_errors 100.0


In [12]:
# посчитаем сколько ошибок делаем на всей выборке
y = df.iloc[:, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[:, [0, 2]].values
pr, hidden = predict(X)
sum(pr-y.reshape(-1, 1))

array([-100.])